# Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd './drive/MyDrive/gaze_estimation'

Mounted at /content/drive
/content/drive/MyDrive/gaze_estimation


In [2]:
import os
import pandas as pd
import numpy as np
import cv2
import random
random.seed(43)
import warnings
warnings.filterwarnings("ignore")

from tqdm.auto import tqdm
from glob import glob
from scipy.io import loadmat

os.getcwd()

'/content/drive/MyDrive/gaze_estimation'

# Data Path

In [3]:
data_path = "../MPIIGaze/Data/Normalized"
mat_path = sorted(glob(os.path.join(data_path,"*","*.mat"))) # p1*으로 수집한 데이터 경로 정의
print(f"Find {len(mat_path)} Data") # 전체 mat 파일 수 출력

Find 521 Data


In [4]:
# https://www.mpi-inf.mpg.de/fileadmin/_processed_/1/4/csm_MPIIGaze_filestruture_3caf5d24b3.png

## Mat to Dataframe

In [5]:
def convert_pose(vector: np.ndarray) -> np.ndarray:
    rot = cv2.Rodrigues(np.array(vector).astype(np.float32))[0]
    vec = rot[:, 2]
    pitch = np.arcsin(vec[1])
    yaw = np.arctan2(vec[0], vec[2])
    return np.array([pitch, yaw]).astype(np.float32)


def convert_gaze(vector: np.ndarray) -> np.ndarray:
    x, y, z = vector
    pitch = np.arcsin(-y)
    yaw = np.arctan2(-x, -z)
    return np.array([pitch, yaw]).astype(np.float32)

In [6]:
eye_loc_list = ["right", "left"] # 눈 위치 정의

# mat 파일 1개씩 작업 수행
datas = []
for idx in tqdm(range(len(mat_path))):
    cur_mat_path = mat_path[idx].replace("\\","/")
    par_id = cur_mat_path.split("/")[-2] # 참가자 id
    day_id = cur_mat_path.split("/")[-1].split(".")[0] # 수집 일자

    # load mat
    mat_data = loadmat(cur_mat_path, squeeze_me=True, struct_as_record=True)['data']
    # 눈 위치별 데이터 추출
    for loc in eye_loc_list:
        eye_loc_data = mat_data[loc].tolist()

        # 이미지 데이터
        image_data = eye_loc_data["image"].tolist()
        if len(image_data.shape) < 3:
            image_data = image_data[np.newaxis,:]

        # Head Pose 데이터
        pose_data = eye_loc_data["pose"].tolist()
        if len(pose_data.shape) < 2:
            pose_data = pose_data[np.newaxis,:]

        # Gaze 데이터
        gaze_data = eye_loc_data["gaze"].tolist()
        if len(gaze_data.shape) < 2:
            gaze_data = gaze_data[np.newaxis,:]


        for i in range(len(image_data)):
            if eye_loc_list == "left":
                image = image_data[i]
                pose = convert_pose(pose_data[i])
                gaze = convert_gaze(gaze_data[i])
            else:
                image = image_data[i][:, ::-1]
                pose = convert_pose(pose_data[i]) * np.array([1, -1])
                gaze = convert_gaze(gaze_data[i]) * np.array([1, -1])

            data_list = [par_id, day_id, loc, image.ravel(), pose[0], pose[1], gaze[0], gaze[1]]
            datas.append(data_list)

# 리스트에 담아두었던 정보들을 DataFrame으로 생성
data_df = pd.DataFrame(columns=["participant_id","day","eye_location","image","head_pitch","head_yaw","gaze_pitch","gaze_yaw"], data=datas)
data_df.head(3)
data_df = data_df.sort_values(by=['participant_id', 'day']).reset_index(drop=True)

  0%|          | 0/521 [00:00<?, ?it/s]

# Save Total Data

In [7]:
save_path = "./mpii_dataset"
if os.path.isdir(save_path) == False:
    os.makedirs(save_path)
save_file = os.path.join(save_path, "mpii_dataset.parquet")
data_df.to_parquet(save_file, engine='pyarrow', index=False)

In [25]:
id_vector = np.asarray(data_df["participant_id"].to_list())
is_right = np.asarray(data_df["eye_location"].to_list()) == "right"
images = np.stack(data_df["image"].to_list())
head_poses = np.asarray(data_df[["head_pitch", "head_yaw"]])
gazes = np.asarray(data_df[["gaze_pitch", "gaze_yaw"]])

In [31]:
os.getcwd()
np.save('./data/MPIIGaze_ids', id_vector)
np.save('./data/MPIIGaze_is_right', is_right)
np.save('./data/MPIIGaze_images', images)
np.save('./data/MPIIGaze_head_poses', head_poses)
np.save('./data/MPIIGaze_gazes', gazes)

# Extract Sampling Data

In [23]:
# indice_list = []

# np.random.seed(42)
# for i in tqdm(range(15)) :
#     if i < 10 :
#         left_indice = np.random.choice(np.where((id_vector == f"p0{i}") & np.invert(is_right))[0], size=1500, replace=False)
#         right_indice = np.random.choice(np.where((id_vector == f"p0{i}") & is_right)[0], size=1500, replace=False)
#         total_indice = np.concatenate([left_indice, right_indice])
#     elif i != 13 :
#         left_indice = np.random.choice(np.where((id_vector == f"p{i}") & np.invert(is_right))[0], size=1500, replace=False)
#         right_indice = np.random.choice(np.where((id_vector == f"p{i}") & is_right)[0], size=1500, replace=False)
#         total_indice = np.concatenate([left_indice, right_indice])
#     else :
#         left_indice = np.random.choice(np.where((id_vector == f"p{i}") & np.invert(is_right))[0], size=2, replace=False)
#         right_indice = np.random.choice(np.where((id_vector == f"p{i}") & is_right)[0], size=2, replace=False)
#         print(np.where(id_vector == f"p{i}")[0])
#         print(left_indice); print(right_indice)
#         total_indice = np.concatenate([np.where(id_vector == f"p{i}")[0], left_indice, right_indice])
#     indice_list.append(total_indice)

# selected_ids = [id_vector[indice] for indice in indice_list]
# selected_images = [images[indice] for indice in indice_list]
# selected_head_poses = [head_poses[indice] for indice in indice_list]
# selected_gazes = [gazes[indice] for indice in indice_list]

  0%|          | 0/15 [00:00<?, ?it/s]

[421320 421321 421322 ... 424313 424314 424315]
[423917 421967]
[422489 422337]
